In [72]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
mistral_api_key = os.environ["MISTRAL_API_KEY"]
tavily_api_key = os.environ["TAVILY_API_KEY"]

## Step by step break down

* Initialize the model and tools
* Initialize graph with state
* Define graph nodes
* Define entry point and graph edges
* Compile the graph
* Execute the graph

## Initialize Mistral LLM Model

In [20]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    mistral_api_key=mistral_api_key,
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    # other params...
)

## Initialize Lang Graph

In [23]:
from langgraph.graph import END, MessageGraph
agent = MessageGraph()

### Define and add nodes in graph.

* Next, we add a single node to the agent, called "node1", which simply calls the LLM with the given input.

In [26]:
agent.add_node("node1", llm)

### Add edge
* We add an edge from this "node1" node to the special string END (__end__). This means that execution will end after the current node.

In [29]:
agent.add_edge("node1", END)

## Set Entry point
* Set "node1" as the entry point to the agent

In [33]:
agent.set_entry_point("node1")

## Compile
* We compile the agent, translating it to low-level pregel operations ensuring that it can be run.

In [36]:
runnable_agent = agent.compile()

## Execute the agent

When we execute the agent:

    LangGraph adds the input message to the state, then passes the state to the entrypoint node, "node1".
    The "node1" node executes, invoking the chat model.
    The chat model returns an AIMessage. LangGraph adds this to the state.
    Execution progresses to the special END value and outputs the final state.
    And as a result, we get a list of two chat messages as output.


In [41]:
from langchain_core.messages import HumanMessage
runnable_agent.invoke(HumanMessage("What is 1+1 ?"))

[HumanMessage(content='What is 1+1 ?', additional_kwargs={}, response_metadata={}, id='c3ee7787-78ed-419b-aab0-ca0b8d1851c6'),
 AIMessage(content='The sum of 1 + 1 is 2. Here it is:\n\n 1\n+1\n____\n 2', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 10, 'total_tokens': 38, 'completion_tokens': 28}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-d3a0fee0-5c26-44a7-95cc-e8c9175e70d5-0', usage_metadata={'input_tokens': 10, 'output_tokens': 28, 'total_tokens': 38})]

## Agent with a router and conditional edges

    Let's allow the LLM to conditionally call a "multiply" node using tool calling.
    We'll recreate our previous agent with an additional "multiply" tool that will take the result of the most recent message, if it is a tool call, and calculate the result.
    We will bind the multiply tool to the OpenAI model to allow the llm to optionally use the tool.

In [49]:
## Write a tool
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def multiply (first: int, sec: int):
    """Multiplies two numbers together"""
    return first*sec



In [53]:
### Create llm and bind multiply tool
llm = ChatMistralAI(
    mistral_api_key=mistral_api_key,
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    # other params...
)
llm_with_tools = llm.bind_tools([multiply])

In [57]:
## Create an agent
conditional_agent = MessageGraph()

# Create first node and set it as entry point
conditional_agent.add_node("node1", llm_with_tools)
conditional_agent.set_entry_point("node1")

# Create second node
tool_node = ToolNode([multiply])
conditional_agent.add_node("multiply", tool_node)

# Create edge 
conditional_agent.add_edge("multiply", END)

## Conditional Router

Using conditional edges, which call a function on the current state and routes execution to a node the function's output:
* If the "node1" node returns a message expecting a tool call, we want to execute the "multiply" node.
* If not, we can just end execution.

In [60]:
from typing import Literal, List
from langchain_core.messages import BaseMessage

def router(state: List[BaseMessage]) -> Literal["multiply", "__end__"]:
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "__end__"

conditional_agent.add_conditional_edges("node1", router)

In [62]:
runnable_conditional_agent = conditional_agent.compile()

In [64]:
runnable_conditional_agent.invoke(HumanMessage("What is 123 * 456?"))

[HumanMessage(content='What is 123 * 456?', additional_kwargs={}, response_metadata={}, id='8d657aa5-6dd1-42b9-a2da-4bd9c9421075'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7L6WrclW9', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"first": 123, "sec": 456}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 86, 'total_tokens': 115, 'completion_tokens': 29}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-fdb45afa-018d-4e3b-bd7c-b630a18757c8-0', tool_calls=[{'name': 'multiply', 'args': {'first': 123, 'sec': 456}, 'id': '7L6WrclW9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 29, 'total_tokens': 115}),
 ToolMessage(content='56088', name='multiply', id='4907c941-8b80-4687-a21e-a6bb5094a82b', tool_call_id='7L6WrclW9')]

In [66]:
runnable_conditional_agent.invoke(HumanMessage("What is your name?"))

[HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={}, id='b7a18e3d-182d-48e2-ae3c-7f6404b03836'),
 AIMessage(content='My name is Assistant.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 79, 'total_tokens': 84, 'completion_tokens': 5}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-b0c185c3-e3b8-4030-bd77-a2202e6b7376-0', usage_metadata={'input_tokens': 79, 'output_tokens': 5, 'total_tokens': 84})]

## Agent With Cycle

    We will use Tavily as online search tool

In [115]:
from langchain_community.tools.tavily_search import TavilySearchResults

online_search_tool = [TavilySearchResults(tavily_api_key = tavily_api_key, max_results=5)]

### 
now wrap these tools in a simple LangGraph ToolNode. This class receives the list of messages (containing tool_calls), calls the tool(s) the LLM has requested to run, and returns the output as new ToolMessage(s).

In [77]:
from langgraph.prebuilt import ToolNode
online_search_tool_node = ToolNode(online_search_tool)

In [81]:
### Create llm and bind tool
llm = ChatMistralAI(
    mistral_api_key=mistral_api_key,
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    # other params...
)
llm_with_tools = llm.bind_tools(online_search_tool)

 Let's now define the state of the agent.
* Each node will return operations to update the state.
* For this example, we want each node to just add messages to the state. Therefore, in this case the state of the agent will be a list of messages. In other projects, the state can be any type.
* We will use a TypedDict with one key (messages) and annotate it so that we always add to the messages key when updating it using the `is always added to` with the second parameter (operator.add).

In [88]:
from typing import TypedDict, Annotated

def add_messages(left: list, right:list):
    """Add-don't-overwrite."""
    return left + right

class AgentState(TypedDict):
    # The `add_messages` function within the annotation defines
    # *how* updates should be merged into the state.
    messages:Annotated[list, add_messages]

## Router

* Let's now define the nodes of this agent:
    * the node responsible for deciding what (if any) actions to take.
    * if the previous node decides to take an action, this second node will then execute that action calling the online search tool.

* We will also define the edges that will interconnect the nodes.
* One will be a Conditional Edge. After the node1 is called, the llm will dedice either:
    * a. Run the online search tool (node2), OR
    * b. Finish

* The second will be a normal Edge: after the online search tool (node2) is invoked, the graph should always return to the node1 to decide what to do next.

In [101]:
from typing import Literal

## Router Function
def should_continue(state: AgentState)->Literal["node2", "__end__"]:
    message = state['messages']
    last_message = message[-1]
    # If the LLM makes a tool call, then we route to the "action" node
    if last_message.tool_calls:
        return "node2"
    # Otherwise, we stop (reply to the user)
    return "__end__"

# Define the function that calls the model
def call_model(state:AgentState):
    messages = state['messages']
    response = llm_with_tools.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [103]:
from langgraph.graph import StateGraph, END

# define a new Graph

workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("node1", call_model)
workflow.add_node("node2", online_search_tool_node)

# Set the entrypoint as `node1`
# This means that this node is the first one called
workflow.set_entry_point("node1")


# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `node1`.
    # This means these are the edges taken after the `node1` node is called.
    "node1",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `online_search_tool` to `node1`.
# This means that after `online_search_tool` is called, `node1` node is called next.
workflow.add_edge('node2', 'node1')


In [105]:
agent_with_cycles = workflow.compile()

In [107]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
agent_with_cycles.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'k302hN2i4', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in sf"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 113, 'total_tokens': 143, 'completion_tokens': 30}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-7e9cc987-72ae-4142-affc-0eae7a2cbe57-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'k302hN2i4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 113, 'output_tokens': 30, 'total_tokens': 143}),
  ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'l

In [111]:
inputs = {"messages": [HumanMessage(content="How about weather of Bangalore whitefield on Saturday, 16th Nov-2024")]}
agent_with_cycles.invoke(inputs)

{'messages': [HumanMessage(content='How about weather of Bangalore whitefield on Saturday, 16th Nov-2024', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'RGxXmBCcW', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather of Bangalore whitefield on Saturday, 16th Nov-2024"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 129, 'total_tokens': 176, 'completion_tokens': 47}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-faef9c3e-f449-423c-842a-59c3fbd59e4f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather of Bangalore whitefield on Saturday, 16th Nov-2024'}, 'id': 'RGxXmBCcW', 'type': 'tool_call'}], usage_metadata={'input_tokens': 129, 'output_tokens': 47, 'total_tokens': 176}),
  ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Bangalore\', \'region\': \'Karn

In [113]:
inputs = {"messages": [HumanMessage(content=" According to Vastu, In which direction should cook face while cokkking? Kitech is in south east direction")]}
agent_with_cycles.invoke(inputs)

{'messages': [HumanMessage(content=' According to Vastu, In which direction should cook face while cokkking? Kitech is in south east direction', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '8jQERu0L0', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "According to Vastu, In which direction should cook face while cokkking? Kitech is in south east direction"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 134, 'total_tokens': 187, 'completion_tokens': 53}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-3366f0c5-9df2-488e-8d1f-c9985f3db351-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'According to Vastu, In which direction should cook face while cokkking? Kitech is in south east direction'}, 'id': '8jQERu0L0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 134, 'output_tokens': 53, 'total_tokens': 187}),
  

In [117]:
inputs = {"messages": [HumanMessage(content=" According to Vastu, In which direction should cook face while cokkking? What is the mitigation plan if cook will not able to face vastu's suggested direction")]}
agent_with_cycles.invoke(inputs)

{'messages': [HumanMessage(content=" According to Vastu, In which direction should cook face while cokkking? What is the mitigation plan if cook will not able to face vastu's suggested direction", additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'p5TgFbkBG', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "According to Vastu, In which direction should cook face while cokkking? What is the mitigation plan if cook will not able to face vastu\'s suggested direction"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 145, 'total_tokens': 209, 'completion_tokens': 64}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-30551ca8-2956-4724-bdf9-b2552af97fa3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': "According to Vastu, In which direction should cook face while cokkking? What is the mitigation plan if cook will not able to face